In [30]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from pprint import pprint
import seaborn as sns
sns.set_theme(style="whitegrid")
import statistics
import math
from scipy import stats
import json
from pandas import json_normalize

In [8]:
base_url_pov = 'https://api.census.gov/data/timeseries/poverty/histpov2'

req1 = requests.get(base_url_pov)
print(req1)  # response 200!

json_data_pov = req1.json()
print(json_data_pov)

<Response [200]>
{'@context': 'https://project-open-data.cio.gov/v1.1/schema/catalog.jsonld', '@id': 'https://api.census.gov/data/timeseries/poverty/histpov2.json', '@type': 'dcat:Catalog', 'conformsTo': 'https://project-open-data.cio.gov/v1.1/schema', 'describedBy': 'https://project-open-data.cio.gov/v1.1/schema/catalog.json', 'dataset': [{'c_dataset': ['timeseries', 'poverty', 'histpov2'], 'c_geographyLink': 'https://api.census.gov/data/timeseries/poverty/histpov2/geography.json', 'c_variablesLink': 'https://api.census.gov/data/timeseries/poverty/histpov2/variables.json', 'c_tagsLink': 'https://api.census.gov/data/timeseries/poverty/histpov2/tags.json', 'c_examplesLink': 'https://api.census.gov/data/timeseries/poverty/histpov2/examples.json', 'c_groupsLink': 'https://api.census.gov/data/timeseries/poverty/histpov2/groups.json', 'c_valuesLink': 'https://api.census.gov/data/timeseries/poverty/histpov2/values.json', 'c_documentationLink': 'https://www.census.gov/developer/', 'c_isTimese

In [34]:
county= 'JEFFERSON'
state_abbr= 'IL'

base_url_wat = ' https://enviro.epa.gov/enviro/efservice/SDW_VIOL_ENFORCEMENT/COUNTYSERVED/'
req2 = requests.get(base_url_wat+ f'{county}/STATE/{state_abbr}/JSON')
print(req2)  # response 200!

json_data_wat = req2.json()

<Response [200]>


In [36]:
# Serializing json 
json_object_wat = json.dumps(json_data_wat)
  
# Writing to .json
with open("water.json", "w") as outfile:
    outfile.write(json_object_wat)

In [39]:
# load data using Python JSON module
with open('water.json','r') as f:
    water_data = json.loads(f.read())
    
# Reading data
water_df = pd.DataFrame(water_data)
water_df.head()

,PWSID,PWSNAME,STATE,COUNTYSERVED,VIOID,CCODE,CNAME,CTYPE,VCODE,VNAME,VTYPE,VIOLMEASURE,ENFACTIONTYPE,ENFACTIONNAME,ENFDATE,COMPPERBEGINDATE,COMPPERENDDATE,SOURCES,DEFINITION,HEALTH_EFFECTS
0,IL0810050,BELLE RIVE,IL,JEFFERSON,2003388,3100,Coliform (TCR),TCR,24,"Monitoring, Routine Minor (TCR)",MR,Monitoring and Reporting,SIF,St Public Notif received,27-JUN-13,01-AUG-12,31-AUG-12,Coliforms are naturally present in the environ...,Coliforms are bacteria that are naturally pres...,Not a health threat in itself; it is used to i...
1,IL0810050,BELLE RIVE,IL,JEFFERSON,2003386,3100,Coliform (TCR),TCR,24,"Monitoring, Routine Minor (TCR)",MR,Monitoring and Reporting,SIA,St Violation/Reminder Notice,16-FEB-11,01-JAN-11,31-JAN-11,Coliforms are naturally present in the environ...,Coliforms are bacteria that are naturally pres...,Not a health threat in itself; it is used to i...
2,IL0810050,BELLE RIVE,IL,JEFFERSON,2003382,3100,Coliform (TCR),TCR,24,"Monitoring, Routine Minor (TCR)",MR,Monitoring and Reporting,SIE,St Public Notif requested,18-FEB-09,01-JAN-09,31-JAN-09,Coliforms are naturally present in the environ...,Coliforms are bacteria that are naturally pres...,Not a health threat in itself; it is used to i...
3,IL0810050,BELLE RIVE,IL,JEFFERSON,2003390,1006,Chloramine,St1 DBP,27,Monitoring and Reporting (DBP),MR,Monitoring and Reporting,SIA,St Violation/Reminder Notice,15-OCT-13,01-JUL-13,30-SEP-13,Water additive used to control microbes,Chloramine (as CI2) is a water additive used t...,"Eye/nose irritation; stomach discomfort, anemia"
4,IL0810050,BELLE RIVE,IL,JEFFERSON,2003382,3100,Coliform (TCR),TCR,24,"Monitoring, Routine Minor (TCR)",MR,Monitoring and Reporting,SIF,St Public Notif received,02-MAR-09,01-JAN-09,31-JAN-09,Coliforms are naturally present in the environ...,Coliforms are bacteria that are naturally pres...,Not a health threat in itself; it is used to i...


In [38]:
info= json.loads(str(json_data_wat))
df=json_normalize(info)
df.head()

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 3 (char 2)